In [0]:
%run ./EDP_Config

In [0]:
df1=spark.sql(f"""select * from {target_database}.incremental_load_cfg where  entity_nm='src_emp'""")
if df1.count() ==1 :
  print('Metadata Present for EMP table')
else:
    print ('Creating Metadata for EMP table')
    userqry="""insert into EDP_DEV.incremental_load_cfg (src_nm,	entity_nm,	wtr_mrk_clm_nm,wtr_mrk_clm_val,src_clm_key,	part_clm_nm, part_clm_val, interval, frequency, clm_dtls, ref_qry,	src_wtr_mrk_qry,	cfg_upd_wtr_mrk_qry,	batch_sz,	ld_seq,	uc_nm,	sch_nm,	mig_flag) select * from (SELECT '',	'src_emp',	'modifieddt',	'1900-01-01T00:00:00.000+0000','Emp_ID'	,'',	'',	1,	1, 
        '(Select Emp_Id, Emp_nm, Dept_Id, createdt, modifieddt, modifieddt as watermark from EDP_source.src_emp where modifieddt > "{}"  and modifieddt <="{}")','',	'select max("{}") from EDP_source."{}"', 'update EDP_DEV.incremental_load_cfg set wtr_mrk_clm_val="{}" where entity_nm="{}"',	'',	null,	'',	'',	''
    )"""
    spark.sql(userqry)

Creating Metadata for EMP table


In [0]:
%sql
select * from edp_dev.incremental_load_cfg

src_nm,entity_nm,wtr_mrk_clm_nm,wtr_mrk_clm_val,src_clm_key,part_clm_nm,part_clm_val,interval,frequency,clm_dtls,ref_qry,src_wtr_mrk_qry,cfg_upd_wtr_mrk_qry,batch_sz,ld_seq,uc_nm,sch_nm,mig_flag
,src_emp,modifieddt,1900-01-01T00:00:00.000+0000,Emp_ID,,,1,1,"(Select Emp_Id, Emp_nm, Dept_Id, createdt, modifieddt, modifieddt as watermark from EDP_source.src_emp where modifieddt > ""{}"" and modifieddt <=""{}"")",,"select max(""{}"") from EDP_source.""{}""","update EDP_DEV.incremental_load_cfg set wtr_mrk_clm_val=""{}"" where entity_nm=""{}""",,null,,,


In [0]:
df1=spark.sql(f"""select * from {target_database}.data_quality_rule where  rule_nm='NULL_CHECK'""")
if df1.count() ==1 :
  print('Rule Present for null values')
else:
    print ('Creating rule for null value')
    null_check_rules_insert = f"""
    INSERT INTO {target_database}.data_quality_rule
    (lob_nm, src_nm, entity_nm, rule_nm, rule_typ, rule_bs_qry, rule_qry, vw_nm, mig_flag)
    VALUES('', '', 'src_emp', 'NULL_CHECK', 'Data quality check','', 'SELECT * from edp_dev.src_emp_brnz WHERE Emp_Id IS NOT NULL AND Emp_nm IS NOT NULL AND Dept_Id IS NOT NULL AND createdt IS NOT NULL AND modifieddt IS NOT NULL', '', '1')
    """
    spark.sql(null_check_rules_insert)

Creating rule for null value


In [0]:
df2 = spark.sql(f"""select * from {target_database}.data_quality_rule where rule_nm='EMPTY_CHECK'""")
if df2.count() == 1:
    print('Rule Present for empty values')
else:
    print('Creating rule for empty value')
    empty_check_rules_insert = f"""
    INSERT INTO {target_database}.data_quality_rule
    (lob_nm, src_nm, entity_nm, rule_nm, rule_typ, rule_bs_qry, rule_qry, vw_nm, mig_flag)
    VALUES('', '', 'src_emp', 'EMPTY_CHECK', 'Data quality check', '', 'SELECT * from edp_dev.src_emp_brnz WHERE Emp_Id != "" OR Emp_nm != "" OR Dept_Id != "" OR createdt != "" OR modifieddt != ""', '', '1')
    """
    spark.sql(empty_check_rules_insert)

Creating rule for empty value


In [0]:
df3 = spark.sql(f"""select * from {target_database}.data_quality_rule where rule_nm='DUPLICATE_CHECK'""")
if df3.count() == 1:
    print('Rule Present for duplicate values')
else:
    print('Creating rule for duplicate value')
    duplicate_check_rules_insert = f"""
    INSERT INTO {target_database}.data_quality_rule
    (lob_nm, src_nm, entity_nm, rule_nm, rule_typ, rule_bs_qry, rule_qry, vw_nm, mig_flag)
    VALUES('', '', 'src_emp', 'DUPLICATE_CHECK', 'Data quality check', '', 'SELECT DISTINCT(*) FROM edp_dev.src_emp_brnz', '', '1')
    """
    spark.sql(duplicate_check_rules_insert)

Creating rule for duplicate value


In [0]:
%sql
SELECT * from edp_dev.data_quality_rule

lob_nm,src_nm,entity_nm,rule_nm,rule_typ,rule_bs_qry,rule_qry,vw_nm,mig_flag
,,src_emp,NULL_CHECK,Data quality check,,SELECT * from edp_dev.src_emp_brnz WHERE Emp_Id IS NOT NULL AND Emp_nm IS NOT NULL AND Dept_Id IS NOT NULL AND createdt IS NOT NULL AND modifieddt IS NOT NULL,,true
,,src_emp,EMPTY_CHECK,Data quality check,,"SELECT * from edp_dev.src_emp_brnz WHERE Emp_Id != """" OR Emp_nm != """" OR Dept_Id != """" OR createdt != """" OR modifieddt != """"",,true
,,src_emp,DUPLICATE_CHECK,Data quality check,,SELECT DISTINCT(*) FROM edp_dev.src_emp_brnz,,true
